In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from run import TrashApp
from model.image_model import TrashModel

Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


In [6]:
trash_app = TrashApp('../data-collection', '../model/first_try.h5', TrashModel())

Camera ready!


In [5]:
while(True):
    trash_app.run()
    sleep(.1)

recyclable


KeyboardInterrupt: 